In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

image = "../assets/car.jpg"
prompt = "What is in the picture?"

In [16]:
# test qwen2VL
from image_text_to_text import Qwen2VLExecutor
tmp = Qwen2VLExecutor("Qwen/Qwen2-VL-7B-Instruct-AWQ")
inputs = {
    "prompt": prompt,
    "image": image,
    "max_output_size": 1000
}
answer = tmp.generate_output(inputs)
print(answer)
print(tmp.get_latency())
print(tmp.get_memory())
# # print(tmp.get_pixels_processed())
# # print(tmp.get_tokens_processed())

We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.30s/it]


['The picture shows a vintage Volkswagen Beetle car parked on a street. The car is painted in a light turquoise color and is positioned in front of a yellow wall with wooden doors. The setting appears to be in a quaint, possibly historical or tourist area, given the style of the building and the car.']
2.879515074
6.510447801217525


In [ ]:

# test florence-2
from executors import FlorenceExecutor
model_name = "microsoft/Florence-2-base"
tmp = FlorenceExecutor(model_name)
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def plot_bbox(image, data):
   # Create a figure and axes
    fig, ax = plt.subplots()

    # Display the image(file str)
    img = plt.imread(image) 
    ax.imshow(img)

    # Plot each bounding box
    for bbox, label in zip(data['bboxes'], data['labels']):
        # Unpack the bounding box coordinates
        x1, y1, x2, y2 = bbox
        # Create a Rectangle patch
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
        # Add the rectangle to the Axes
        ax.add_patch(rect)
        # Annotate the label
        plt.text(x1, y1, label, color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

    # Remove the axis ticks and labels
    ax.axis('off')

    # Show the plot
    plt.savefig('florence_test.jpg')

# Caption
task_prompt = '<CAPTION>'
inputs = {
    "prompt": task_prompt,
    "image": image,
    "text": "",
    "max_output_size": 200
}
answer = tmp.generate_output(inputs)
print(answer)
task_prompt = '<DETAILED_CAPTION>'
answer = tmp.generate_output(task_prompt, image)
print(answer)
task_prompt = '<MORE_DETAILED_CAPTION>'
answer = tmp.generate_output(task_prompt, image)
print(answer)

# Object detection
task_prompt = '<OD>'
answer = tmp.generate_output(task_prompt, image)
print(answer)
plot_bbox(image, answer['<OD>'])

task_prompt = "<DENSE_REGION_CAPTION>"
answer = tmp.generate_output(task_prompt, image)
print(answer)
plot_bbox(image, answer['<DENSE_REGION_CAPTION>'])

# ocr
image = "../assets/cuda.jpg"
task_prompt = '<OCR>'
answer = tmp.generate_output(task_prompt, image)
print(answer)
task_prompt = '<OCR_WITH_REGION>'
answer = tmp.generate_output(task_prompt, image)
print(answer)

# More Detailed Caption + Phrase Grounding 
task_prompt = '<MORE_DETAILED_CAPTION>'
results = tmp.generate_output(task_prompt, image)
text_input = results[task_prompt]
task_prompt = '<CAPTION_TO_PHRASE_GROUNDING>'
results = tmp.generate_output(task_prompt, image, text_input)
results['<MORE_DETAILED_CAPTION>'] = text_input
print(results)
plot_bbox(image, results['<CAPTION_TO_PHRASE_GROUNDING>'])

In [ ]:
#  test llava
from executors import LLaVAExecutor
model_name = "llava-hf/llava-1.5-13b-hf"
tmp = LLaVAExecutor(model_name)
inputs = {
    "prompt": prompt,
    "image": image,
    "max_output_size": 1000
}
results = tmp.generate_output(inputs)
print(results)
print("Inference latency is：", tmp.get_latency(), "s")